## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(23)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [9]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [10]:
# collect
# rdd.collect()
# this will cause your system to crash with a large dataset. 

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.70314498e-01, 1.37543088e+00, 2.65363346e+00, 8.95907760e-02,
       9.25328867e-01, 3.08149552e-01, 5.02022643e+00, 3.96024792e+00,
       8.14795378e+00, 2.84436645e+00, 2.68848144e-01, 9.94117705e+00,
       4.40117773e+00, 6.62123060e+00, 4.24715743e+00, 1.27629472e-01,
       1.26147442e+01, 3.77070122e+00, 1.23421083e+01, 5.96934860e+00,
       7.21270212e+00, 1.27638994e+01, 7.80602200e+00, 2.39954197e+01,
       4.90822468e+00, 1.11642513e+01, 1.85621639e+01, 1.15603472e+00,
       1.36703422e+01, 2.09818045e+00, 2.75964071e+01, 1.95139543e+01,
       8.92645626e+00, 2.12804346e+01, 2.94638049e+01, 1.45649295e+01,
       5.61347268e+00, 3.71646265e+01, 5.57469002e+00, 1.00515095e+01,
       3.05089723e+01, 1.06408546e+01, 1.54786637e+01, 3.27552530e+01,
       3.12611466e+01, 3.88275517e+00, 2.72614861e+01, 1.79262774e+01,
       1.95872078e+01, 4.78743481e+01, 2.45185352e+01, 2.76333326e+01,
       1.09959441e+00, 1.79702204e+01, 1.60810595e+01, 1.05577823e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [12]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.6703144979949379,
 1.3754308768027401,
 2.6536334578040552,
 0.08959077600311627]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [14]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.6166893381553429,
 1.265396406658521,
 2.441342781179731,
 0.08242351392286697,
 0.8513025573468408,
 0.28349758798660857,
 4.618608317801708,
 3.6434280860817996,
 7.4961174797577375,
 2.6168171318104756]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [15]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [16]:
discounted.take(10)

[0.5550204043398086,
 1.138856765992669,
 2.197208503061758,
 0.07418116253058027,
 0.7661723016121568,
 0.2551478291879477,
 4.156747486021537,
 3.27908527747362,
 6.746505731781964,
 2.355135418629428]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [17]:
price_items.map(sales_tax).map(lambda x: x * 0.9).top(15)

[778.689410927506,
 777.577535462069,
 743.9962022340884,
 742.0936093016128,
 738.9325366463918,
 736.6003695018046,
 732.3493487533432,
 724.7277748745835,
 714.3965663720161,
 708.170664142305,
 700.7996849398136,
 695.2136696895966,
 691.2304179044888,
 690.2273228312489,
 689.6628348146451]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [18]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [19]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.6703144979949379, 0.5550204043398086), (1.3754308768027401, 1.138856765992669), (2.6536334578040552, 2.197208503061758), (0.08959077600311627, 0.07418116253058027), (0.9253288666813486, 0.7661723016121568), (0.30814955215935713, 0.2551478291879477), (5.0202264323931605, 4.156747486021537), (3.9602479196541296, 3.27908527747362), (8.147953782345366, 6.746505731781964), (2.8443664476200823, 2.355135418629428)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [21]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.6703144979949379, 0.5550204043398086, 1.3754308768027401, 1.138856765992669, 2.6536334578040552, 2.197208503061758, 0.08959077600311627, 0.07418116253058027, 0.9253288666813486, 0.7661723016121568]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [22]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

283

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [23]:
selected_items.reduce(lambda x,y: x + y)

127349.82771930752

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [24]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(3, 314.834820809459),
 (15, 327.22180553250547),
 (26, 331.00413163189313),
 (9, 344.23630948276025),
 (24, 310.839376092282),
 (28, 313.23862115907656),
 (8, 339.09330661000547)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [25]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(10, 1864.729125805509),
 (20, 2408.808306993332),
 (40, 394.0171825648616),
 (30, 2333.4039584798543),
 (50, 2883.035363629682),
 (41, 3703.6124798957258),
 (31, 2560.089834584736),
 (11, 2065.1300342351387),
 (21, 1385.0189500184517),
 (1, 5235.8995248680585)]

In [26]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending = False).collect()

[(1, 5235.8995248680585),
 (48, 4545.616517861742),
 (27, 4037.815495631615),
 (49, 3959.2754223561024),
 (42, 3954.1190362949915),
 (32, 3830.398507752494),
 (25, 3806.527636665618),
 (41, 3703.6124798957258),
 (35, 3559.914599945754),
 (22, 3479.984847364666),
 (26, 3411.2566071952338),
 (39, 3298.26742492163),
 (7, 3231.0113482906063),
 (2, 3165.5881885469944),
 (6, 3095.1866586176766),
 (17, 2974.9144415737287),
 (36, 2914.9621815851137),
 (37, 2901.8761383263386),
 (50, 2883.035363629682),
 (29, 2875.1003089062847),
 (16, 2786.809989716252),
 (45, 2718.081812247533),
 (24, 2639.266802729663),
 (28, 2569.6769562476165),
 (31, 2560.089834584736),
 (4, 2488.7145845685905),
 (8, 2465.181857873372),
 (20, 2408.808306993332),
 (47, 2373.8006324538337),
 (30, 2333.4039584798543),
 (15, 2275.012126348074),
 (33, 2173.190856096218),
 (43, 2128.736631726347),
 (5, 2125.1251335088136),
 (11, 2065.1300342351387),
 (3, 1969.65303182185),
 (13, 1884.0823421484201),
 (10, 1864.729125805509),
 (3

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [27]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv: kv[1], reverse=True)

[(7, 10),
 (26, 10),
 (11, 9),
 (21, 8),
 (46, 8),
 (34, 8),
 (31, 7),
 (9, 7),
 (6, 7),
 (48, 7),
 (27, 7),
 (17, 7),
 (22, 7),
 (40, 7),
 (4, 7),
 (14, 7),
 (8, 7),
 (23, 7),
 (49, 6),
 (28, 6),
 (18, 6),
 (47, 6),
 (33, 6),
 (35, 6),
 (39, 6),
 (44, 6),
 (1, 6),
 (24, 6),
 (43, 6),
 (45, 6),
 (38, 5),
 (3, 5),
 (36, 5),
 (42, 5),
 (19, 5),
 (12, 5),
 (10, 4),
 (5, 4),
 (13, 4),
 (25, 4),
 (41, 4),
 (29, 4),
 (15, 4),
 (20, 4),
 (50, 3),
 (32, 3),
 (16, 2),
 (37, 2),
 (2, 1),
 (30, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
